In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
csv_files = ['ap_2010.csv','class_size.csv','demographics.csv', 'graduation.csv', 'hs_directory.csv', 'sat_results.csv']
text_files = ['survey_all.txt', 'survey_d75.txt']

##### creating a dictionary of all the datasets

In [3]:
data = {}
for file in csv_files:
    file_name = file.split('.')
    data[file_name[0]]= pd.read_csv(file)

##### correcting "survey" data set and add it to the dictionary

In [4]:
#read the file with its encoding
survey_all = pd.read_csv(text_files[0], sep = '\t', encoding = 'cp1252')
survey_d75 = pd.read_csv(text_files[1], sep = '\t', encoding = 'cp1252')

#merging two data set to one
survey = pd.concat([survey_all, survey_d75], axis = 0)

#add survey to the data dictionary by choosing the custom and useful columns
survey['DBN'] = survey['dbn']
custom_columns = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]
survey = survey[custom_columns].copy()
data['survey'] = survey

##### correcting "class_size" data set by adding DBN column

In [5]:
#function to add a zero to left side of the numbers in CSD column
def correct_CSD(column):
    CSD = str(column).zfill(2) 
    column = CSD
    return column
data['class_size']['CSD'] = data['class_size']['CSD'].apply(correct_CSD)

#create DBN column
data['class_size']['DBN'] = data['class_size']['CSD'] + data['class_size']['SCHOOL CODE']

#move DBN column to the front
cols = list(data['class_size'])
cols.insert(0, cols.pop(cols.index('DBN')))
data['class_size'] = data['class_size'].loc[:, cols]

##### correcting "hs_directory" data set

In [6]:
data['hs_directory'].rename(columns = {'dbn':'DBN'}, inplace = True)

##### correcting "ap_2010" data types to float

In [7]:
for column in data['ap_2010'].columns[2:]:
    data['ap_2010'][column] = pd.to_numeric(data['ap_2010'][column], errors = 'coerce')

##### extracting and adding latitude and longitude columns to "hs_directory" for further use

In [8]:
#function to extract the latitude and longitude from column "Location 1"
def add_coordinates (column):
    if column == "NaN":
        return None
    else:
        location = re.findall('\(.+\)',column)
        location = location[0].split(',')
        latitude = float(location[0][1:])
        longitude = float(location[1][:-1])
        coordinates = [latitude, longitude]
        return coordinates

#apply the function and assign the results to coordinates and create two list from it to add to the main dataframe    
coordinates = data['hs_directory']['Location 1'].apply(add_coordinates)
latitude = []
longitude = []
for cor in coordinates:
    latitude.append(cor[0])
    longitude.append(cor[1])

#add the new columns to DataFrame
data['hs_directory']['latitude'] = latitude
data['hs_directory']['longitude'] = longitude

##### condensing the 'class_size' by deleting the rows that are not useful i.e. not for high school students or 

In [9]:
#finding all the possible values for 'GRADE' column
data['class_size']['GRADE '].unique()
#we just want the '09-12' for high school students, so delete the other rows if the GRADE is not '09-12'
data['class_size'] = data['class_size'][data['class_size']['GRADE '] == '09-12']

#finding the possible values for Program type and their number of repetition
data['class_size']['PROGRAM TYPE'].unique()
        
#GEN ED is the most popular TYPE so delete the rest
data['class_size'] = data['class_size'][data['class_size']['PROGRAM TYPE'] == 'GEN ED']

#grouping and aggregating based on the DBN and average of other columns
data['class_size'] =  data['class_size'].groupby('DBN', as_index = False).mean()

##### condensing the 'demographics' by deleting the rows other than the schoolyear of 20112012 which is the same as the sat results year

In [10]:
data['demographics'] = data['demographics'][data['demographics']['schoolyear'] == 20112012]

##### condensing the 'graduation' by keeping only Total Cohort and  2006 

In [11]:
data['graduation'] = data['graduation'][data['graduation']['Demographic']== 'Total Cohort'];
data['graduation'] = data['graduation'][data['graduation']['Cohort']== '2006'];

##### condensing the 'ap_2010' by deleting one row that was excess 

In [12]:
#find the DBN that is repeated more than one row
for (index, value) in (data['ap_2010']['DBN'].value_counts()).iteritems():
    if value >1:
        print(index, value)
        
#check those rows        
data['ap_2010'][data['ap_2010']['DBN'] == '04M610']

#delete the row that was extra by knowing its index
data['ap_2010'] = data['ap_2010'].drop(52)

04M610 2


##### summing up all the SAT sections to "sat_score"

In [13]:
data['sat_results']['SAT Critical Reading Avg. Score'] = pd.to_numeric(data['sat_results']['SAT Critical Reading Avg. Score'],errors = 'coerce')
data['sat_results']['SAT Math Avg. Score'] = pd.to_numeric(data['sat_results']['SAT Math Avg. Score'],errors = 'coerce')
data['sat_results']['SAT Writing Avg. Score'] = pd.to_numeric(data['sat_results']['SAT Writing Avg. Score'],errors = 'coerce')
data['sat_results']['sat_score'] = data['sat_results']['SAT Critical Reading Avg. Score'] \
                                 + data['sat_results']['SAT Math Avg. Score'] \
                                 + data['sat_results']['SAT Writing Avg. Score']

##### merging all the dataframes

In [14]:
combined = data['sat_results']
combined = combined.merge(data['ap_2010'], how = 'left', on = 'DBN')
combined = combined.merge(data['graduation'], how = 'left', on = 'DBN')
combined = combined.merge(data['class_size'], how = 'inner', on = 'DBN')
combined = combined.merge(data['demographics'], how = 'inner', on = 'DBN')
combined = combined.merge(data['survey'], how = 'inner', on = 'DBN')
combined = combined.merge(data['hs_directory'], how = 'inner', on = 'DBN')

##### filling the NaN values, and droping two repeated columns

In [15]:
combined = combined.fillna(combined.mean())
combined = combined.fillna(0)
combined.drop('SchoolName', axis = 1, inplace = True)
combined.drop('School Name', axis = 1, inplace = True)

##### add school_dist column for further analysis

In [16]:
combined['school_dist'] = combined['DBN'].apply(lambda x : x[0:2] )